In [1]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import IPython.display as ipd
from tqdm import tqdm

In [2]:
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

2023-11-29 10:20:37.447533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 10:20:37.572784: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 10:20:38.034095: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-29 10:20:38.034183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
data_path = "/home/bmis/Documents/AI-Workspace/ALS/bmis_data/All"

control_path = 'Control'
als_with_dysarthria_path = 'ALSwDysarthria'
als_without_dysarthria_path = 'ALSwoDysarthria'

control_dir = os.path.join(data_path, control_path)
control = os.listdir(control_dir)[0]

als_with_dir = os.path.join(data_path, als_with_dysarthria_path)
als_with_dys = os.listdir(als_with_dir)[1]

als_without_dir = os.path.join(data_path, als_without_dysarthria_path)
als_without_dys = os.listdir(als_without_dir)[1]

In [4]:
def extract_MFCC_features(audio_path):
    audio, sample_rate = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=128)
    #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
    return mfccs_scaled


def get_MFCC_data(data_path, label):
    data = []
    labels = []
    for file in tqdm(os.listdir(data_path)):
        try:
            feature = extract_MFCC_features(os.path.join(data_path, file))
            data.append(feature)
            labels.append(label)
        except:
            print("Error encountered while parsing file: ", file)
            continue
    return data, labels

## 2 Class ALS CLASSIFICATION

In [5]:
control_data, control_labels = get_MFCC_data(control_dir, 0)
als_with_data, als_with_labels = get_MFCC_data(als_with_dir, 1)
als_without_data, als_without_labels = get_MFCC_data(als_without_dir, 1)

X = np.concatenate([control_data, als_with_data, als_without_data], axis=0)
y = np.concatenate([control_labels, als_with_labels, als_without_labels], axis=0)

X = X.reshape(-1,16,8,1)

X,y = shuffle(X, y, random_state=42)

100%|██████████| 176/176 [00:03<00:00, 48.27it/s]


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(16, 8, 1)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

2023-11-29 10:20:57.088339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-29 10:20:57.125975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-11-29 10:20:57.125994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-11-29 10:20:57.126359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 8, 32)         320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 4, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 4, 64)          18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 2, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 2, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 1, 64)         0

In [7]:
num_folds = 10
fold_no = 1
kfold = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []
loss_per_fold = []

# Model Hyperparameters Turning
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1,
                                                 restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('cnn_two_class_model.h5', monitor='loss', verbose=1,
                                                      save_best_only=True)

for train, test in kfold.split(X, y):
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('---------------------------------------------------')

    history = model.fit(X[train], y[train], epochs=100, verbose=1,
                        callbacks=[earlystopping, model_checkpoint])

    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    ###
    y_pred = model.predict(X[test])
    y_pred = np.round(y_pred).astype(int)

    precision = precision_score(y[test], y_pred)
    recall = recall_score(y[test], y_pred)
    f1 = f1_score(y[test], y_pred)



    # Plot the confusion matrix

    #print('***************************************************')
    #print(f'Precision: {precision} -> Recall: {recall} -> F1: {f1}')
    #print('***************************************************')
    ##
    accuracy_per_fold.append(scores[1] * 100)
    precision_per_fold.append(precision)
    recall_per_fold.append(recall)
    f1_per_fold.append(f1)
    loss_per_fold.append(scores[0])
    
    """"
    if fold_no == 9:

        class_names = ['Control', 'ALS']
        confusion = confusion_matrix(y[test], y_pred)
        plot_confusion_matrix(conf_mat=confusion, figsize=(12,8), class_names=class_names, hide_ticks=True, cmap=plt.cm.Blues)

    """
    fold_no += 1

---------------------------------------------------
Epoch 1/100
14/19 [=====================>........] - ETA: 0s - loss: 1.1178 - accuracy: 0.6183
Epoch 1: loss improved from inf to 0.98170, saving model to cnn_two_class_model.h5
19/19 [==============================] - 0s 5ms/step - loss: 0.9817 - accuracy: 0.6528
Epoch 2/100
14/19 [=====================>........] - ETA: 0s - loss: 0.5396 - accuracy: 0.7277
Epoch 2: loss improved from 0.98170 to 0.53679, saving model to cnn_two_class_model.h5
19/19 [==============================] - 0s 5ms/step - loss: 0.5368 - accuracy: 0.7329
Epoch 3/100
14/19 [=====================>........] - ETA: 0s - loss: 0.5661 - accuracy: 0.7254
Epoch 3: loss did not improve from 0.53679
19/19 [==============================] - 0s 4ms/step - loss: 0.5429 - accuracy: 0.7346
Epoch 4/100
14/19 [=====================>........] - ETA: 0s - loss: 0.4766 - accuracy: 0.7790
Epoch 4: loss improved from 0.53679 to 0.46876, saving model to cnn_two_class_model.h5
19/19 [

In [8]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
    print(f'> Fold {i+1} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('-----------------------------------------------')

Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.3646876811981201 - Accuracy: 94.02984976768494%
> Fold 1 - Precision: 0.9148936170212766 - Recall: 1.0 - F1: 0.9555555555555556
-----------------------------------------------
> Fold 2 - Loss: 0.0003660108195617795 - Accuracy: 100.0%
> Fold 2 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 3 - Loss: 4.4458015509007964e-06 - Accuracy: 100.0%
> Fold 3 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 4 - Loss: 0.0069983648136258125 - Accuracy: 100.0%
> Fold 4 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 5 - Loss: 2.208855448770919e-06 - Accuracy: 100.0%
> Fold 5 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 6 - Loss: 7.760526386846323e-06 - Accuracy: 100.0%
> Fold 6 - Precision: 1.0 - Recall: 1.0 - F1: 1.0


## 3 Class ALS CLASSIFICATION

In [9]:
control_data, control_labels = get_MFCC_data(control_dir, 0)
als_with_data, als_with_labels = get_MFCC_data(als_with_dir, 1)
als_without_data, als_without_labels = get_MFCC_data(als_without_dir, 2)


X = np.concatenate([control_data, als_with_data, als_without_data], axis=0)
y = np.concatenate([control_labels, als_with_labels, als_without_labels], axis=0)
X = np.array(X)


# Data Cleaning

X[X == None] = np.nan
X = np.nan_to_num(X, nan=0.0)
df = pd.DataFrame(X)
df.fillna(0.0, inplace=True)
X = df.to_numpy()

X = X.reshape(-1,16,8,1)

X,y = shuffle(X, y, random_state=42)

100%|██████████| 176/176 [00:03<00:00, 52.42it/s]


In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(16, 8, 1)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
# Print model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 16, 8, 32)         320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 8, 4, 32)         0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 4, 64)          18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 4, 2, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 2, 64)          36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 2, 1, 64)        

In [12]:
num_folds = 10
fold_no = 1
kfold = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []
loss_per_fold = []

# Model Hyperparameters Turning
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1,
                                                 restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('cnn_three_class_model.h5', monitor='loss', verbose=1,
                                                      save_best_only=True)

for train, test in kfold.split(X, y):
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('---------------------------------------------------')

    history = model.fit(X[train], y[train], epochs=100, verbose=1,
                        callbacks=[earlystopping, model_checkpoint])

    scores = model.evaluate(X[test], y[test], verbose=0)


    y_pred = model.predict(X[test])
    y_pred = np.argmax(y_pred, axis=-1)
    precision = precision_score(y[test], y_pred, average='weighted')
    recall = recall_score(y[test], y_pred, average='weighted')
    f1 = f1_score(y[test], y_pred, average='weighted')


    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold.append(scores[1] * 100)
    precision_per_fold.append(precision)
    recall_per_fold.append(recall)
    f1_per_fold.append(f1)
    loss_per_fold.append(scores[0])
    
    
    """"
    if fold_no == 9:

        class_names = ['Control', 'ALS_with_dysarthria', 'ALS_without_dysarthria']
        confusion = confusion_matrix(y[test], y_pred)
        plot_confusion_matrix(conf_mat=confusion, figsize=(12,8), class_names=class_names, hide_ticks=True, cmap=plt.cm.Blues)
     """
    
    fold_no += 1

---------------------------------------------------
Epoch 1/100
10/19 [==============>...............] - ETA: 0s - loss: 0.5559 - accuracy: 0.9000    
Epoch 1: loss improved from inf to 0.38125, saving model to cnn_three_class_model.h5
19/19 [==============================] - 1s 26ms/step - loss: 0.3813 - accuracy: 0.9232
Epoch 2/100
15/19 [======================>.......] - ETA: 0s - loss: 0.0747 - accuracy: 0.9750
Epoch 2: loss improved from 0.38125 to 0.06078, saving model to cnn_three_class_model.h5
19/19 [==============================] - 0s 17ms/step - loss: 0.0608 - accuracy: 0.9800
Epoch 3/100
14/19 [=====================>........] - ETA: 0s - loss: 0.0103 - accuracy: 1.0000
Epoch 3: loss improved from 0.06078 to 0.00904, saving model to cnn_three_class_model.h5
19/19 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 4/100
14/19 [=====================>........] - ETA: 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 4: loss improved from 0.00904 to

In [13]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
    print(f'> Fold {i+1} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('-----------------------------------------------')

Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.9925483465194702 - Accuracy: 83.58209133148193%
> Fold 1 - Precision: 0.8490650197289414 - Recall: 0.835820895522388 - F1: 0.8339810250258011
-----------------------------------------------
> Fold 2 - Loss: 0.003091977210715413 - Accuracy: 100.0%
> Fold 2 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 3 - Loss: 2.95710597129073e-05 - Accuracy: 100.0%
> Fold 3 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 4 - Loss: 4.324359179008752e-05 - Accuracy: 100.0%
> Fold 4 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 5 - Loss: 0.009948848746716976 - Accuracy: 100.0%
> Fold 5 - Precision: 1.0 - Recall: 1.0 - F1: 1.0
-----------------------------------------------
> Fold 6 - Loss: 0.002751043066382408 - Accuracy: 100.0%
> Fold 6 - Precision: 1.0 - Recall: 1.0 -